In [1]:
suppressMessages(require(tidyverse))
suppressMessages(library(DESeq2))

suppressMessages(require(pheatmap))
suppressMessages(require(RColorBrewer))
suppressMessages(require(dendextend))
suppressMessages(require(ComplexHeatmap))
suppressMessages(require(circlize))

suppressMessages(library(data.table))

suppressMessages(library(rstatix))
suppressMessages(library(ggpubr))

suppressMessages(library(grid))
suppressMessages(library(gridExtra))

source("../0_support-files/theme_CRP-MISC.R")

annotation <- fread(file="../0_support-files/gencode.biotype.name.key.tsv") %>% mutate(gene_key = paste0(gene_id ,"_", gene_name))



In [2]:
covid_control <- read.delim("./wb_covid_ctrls.txt") %>% mutate(gene_id = paste0(gene,"_",gene_name))
misc_control <- read.delim("./wb_misc_ctrls.txt") %>% mutate(gene_id = paste0(gene,"_",gene_name))
misc_covid <- read.delim("./wb_misc_covid.txt") %>% mutate(gene_id = paste0(gene,"_",gene_name))


### Read in metadata
meta_data_all <- read.csv("../1_sample-data/STable7_wbrna-samples.csv")
meta_data_all[is.na(meta_data_all$ivig_rel_samp),"ivig_rel_samp"] <- "noivig"

### Read ftcount matrix
raw_ftcount <- read.delim("../1_sample-data/wbrna_ftcounts.txt",row.names = 1)
nrow(raw_ftcount)

### Read gene list to remove
gene.list <- read.delim("../0_support-files/genelist.remove.tsv",col.names = c("type,","ENSMBL","gene_symbol"))
gene.ids <- gsub("\\..*","",rownames(raw_ftcount))

### Subset count matrix
exclude.idx <- gene.ids %in% gene.list[,2]
raw_ftcount = raw_ftcount[!exclude.idx,]          

nrow(raw_ftcount)

raw_ftcount <- raw_ftcount[,colSums(raw_ftcount) > 0]

cpm_ftcount <- edgeR::cpm(raw_ftcount)
head(cpm_ftcount)

[1] 60708

[1] 56622

,PV156,PV157,PV158,PV159,PV160,PV162,PV163,PV164,PV166,PV167,⋯,PV90,PV91,PV92,PV93,PV94,PV95,PV96,PV97,PV98,PV99
ENSG00000000419.14_DPM1,100.94624,11.44980,12.499102,17.346475,28.274329,18.399866,25.73987,11.817701,11.587262,22.017931,⋯,27.87094,37.002291,26.3019325,27.007076,43.689324,35.397525,18.204483,28.6788406,24.777141,20.7923057
ENSG00000000457.14_SCYL3,106.25920,30.82638,30.206162,10.407885,42.411493,24.533155,27.71986,9.762449,14.346134,18.414997,⋯,0.00000,13.662384,6.3124638,9.002359,7.418942,16.592590,12.136322,11.8456081,24.777141,3.9604392
ENSG00000000460.17_C1orf112,0.00000,10.56905,13.540693,8.673238,16.964597,16.764322,37.61981,9.762449,13.242585,15.212389,⋯,27.87094,4.554128,3.3315781,0.000000,9.067596,4.424691,6.935041,6.2345306,1.935714,19.8021959
ENSG00000000938.13_FGR,143.44992,202.57339,158.321954,375.551184,216.500575,167.234337,104.93948,209.121929,239.470076,393.920809,⋯,854.70880,532.548353,373.1367492,180.047172,514.379970,196.161286,243.593322,360.3558670,712.342810,141.5857004
ENSG00000000971.16_CFH,0.00000,0.00000,6.249551,0.000000,0.807838,2.862201,0.00000,0.000000,3.862421,0.400326,⋯,0.00000,1.423165,0.8767311,0.000000,0.000000,2.949794,3.467521,0.6234531,1.935714,0.9901098
ENSG00000001036.14_FUCA2,10.62592,29.06488,16.665469,2.601971,11.309732,12.675463,39.59980,6.679570,15.449682,18.815323,⋯,0.00000,13.947017,13.6770049,9.002359,13.189230,8.111933,11.269442,13.7159673,22.067141,8.9109881


---
## Heatmaps

#### _Functions_

In [24]:
create_heatmap <- function(res,groups,cpm,meta_data) {
    
    ANNOTATIONS = c("Diagnosis","severity",'group','ivig_rel_samp')

    
    ###----------------------------------
    ## Extract significant genes
    sig_genes <- data.frame(res) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% pull(gene_id)

    ###----------------------------------
    ## Subset metadata 
    metadata <- meta_data %>% 
        filter((Diagnosis %in% all_of(groups)) & (timepoint %in% c("Not-hospitalized","acute"))) %>% 
        column_to_rownames(var = "wbrna_sample_id") %>%  
        mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
        select(all_of(ANNOTATIONS))

    colnames(metadata) <- c("Diagnosis","Severity","group","IVIG")

    ###----------------------------------
    ## Subset count matrix
    mat <- data.frame(cpm) %>% 
            filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
            select(all_of(rownames(metadata))) %>% 
            as.matrix()

    mat <- t(scale(t(mat)))

    ###----------------------------------
    ## Colors
    color = colorRampPalette(c("blue","yellow"))(50)
    breaksList = seq(-2, +2, length = 51)

    #color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')
    
    my_colour <- list(
    Diagnosis = c('COVID-19' = '#F0484E', 'MIS-C' = '#5CB2EB', 'Control_Non-inflammatory' = '#FBE77C'), #"MISC_acute_validation" = '#00FFFF'),
    Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
    group = c("validation" = "orange", "discovery" = "light blue", "UCSF" = "light blue"),
    IVIG = c("after" = "maroon", "before" = "darkseagreen2", "concurrent with" = "dark blue","noivig"="white")#, "forest green"
    # PCR = c("1" = "green", "2" = "blue"),
    # Antibody =c("1" = "green", "2" = "blue")
    )

    ###----------------------------------
    ## Plot

    # pdf("./tmp.pdf")
    heatmap_plt <- pheatmap(mat,

             # Colors
             col=color,
             breaks=breaksList, 
             annotation_col=metadata,
             annotation_colors=my_colour,
             na_col = "#FFFFFF",

             # Fonts
             show_colnames=F,
             show_rownames=F,
             fontsize=12,
             fontsize_col=3,
             annotation_names_col=F,
             annotation_names_row=F,

             # Clustering

             clustering_distance_cols="correlation",
             clustering_distance_rows="correlation", 
    #          clustering_distance_cols="euclidean",
    #          clustering_distance_rows="euclidean",
    #          cluster_cols=hc,
    #          cluster_rows=hr,
    #          clustering_distance_rows="euclidean",
    #          clustering_method="complete",
             treeheight_row=0,
            treeheight_col= 15,

             # Misc.
             border_color=NA,
            legend=FALSE,
            annotation_legend=FALSE
            ) 
    # dev.off()




    return(heatmap_plt)

        }

In [25]:
SIG_THRESH = 0.01
FC_THRESH = 1.5

# SIG_THRESH = 0.05
# FC_THRESH = 0

ANNOTATIONS <- c("Diagnosis","severity")

In [26]:

covid_control_plt <- create_heatmap(covid_control,c("COVID-19","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)


misc_control_plt <- create_heatmap(misc_control,c("MIS-C","Control_Non-inflammatory"),cpm_ftcount,meta_data_all)


misc_covid_plt <- create_heatmap(misc_covid,c("MIS-C","COVID-19"),cpm_ftcount,meta_data_all)

In [27]:
WIDTH = 1.6
HEIGHT = 2.6

RES = 300

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "COVID19"
       
png(file="plots/panelB_covid-control.png",
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(covid_control_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "MISC"

png(file="plots/panelB_misc-control.png",
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_control_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

##------------------------------------
# SAVE HEATMAP OBJECT

prefix = "MISC_COVID19"

png(file="plots/panelB_misc-covid.png",
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

# pdf(file=paste0("plots/",prefix,".heatmap.cfrna.pdf"),
#         width=WIDTH,height=HEIGHT, paper="special", bg="transparent",
#         fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

draw(misc_covid_plt,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()

png 
  2

png 
  2

png 
  2

In [17]:
res1=covid_control
res2=misc_control
res3=misc_covid
cpm=cpm_ftcount
meta_data= meta_data_all
groups = c(1,3,57,58,33,35,2,4,34,36,31)
    
ANNOTATIONS = c("Diagnosis","severity",'group')


###----------------------------------
## Extract significant genes

N = 300

sig_genes1 <- data.frame(res1) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% arrange(desc(abs(log2FoldChange))) %>%
head(N) %>% pull(gene_id)

sig_genes2 <- data.frame(res2) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% arrange(desc(abs(log2FoldChange))) %>%
head(N) %>% pull(gene_id)

sig_genes3 <- data.frame(res3) %>% filter(padj < SIG_THRESH & abs(log2FoldChange) > FC_THRESH) %>% arrange(desc(abs(log2FoldChange))) %>%
head(N) %>% pull(gene_id)

sig_genes <- union(sig_genes1,union(sig_genes2,sig_genes3))


###----------------------------------
## Subset metadata 
metadata <- meta_data %>% 
    filter(sample_group %in% all_of(groups)) %>% 
    mutate(sample_group = factor(sample_group, levels = all_of(groups))) %>% 
    arrange(sample_group) %>% 
    column_to_rownames(var = "wbrna_sample_id") %>%  
    mutate(severity = ifelse(grepl("ontrol",severity),NA,severity)) %>%
    select(all_of(ANNOTATIONS))

colnames(metadata) <- c("Diagnosis","Severity","group")

###----------------------------------
## Subset count matrix
mat <- data.frame(cpm) %>% 
        filter(row.names(cpm) %in% all_of(sig_genes)) %>% 
        select(all_of(rownames(metadata))) %>% 
        as.matrix()

mat <- t(scale(t(mat)))

mat <- mat[,rownames(metadata)]

###----------------------------------
## Colors
color = colorRampPalette(c("blue","yellow"))(50)
breaksList = seq(-2, +2, length = 51)

#color_groups = c('COVID-19\ndiscovery' = '#c1272d', 'MIS-C\ndiscovery' = '#0000a7', 'Control_Non-inflammatory\ndiscovery' = '#eecc16',"MIS-C\nvalidation" = '#008176')

my_colour <- list(
Diagnosis = c('COVID-19' = '#F0484E', 'MIS-C' = '#5CB2EB', 'Control_Non-inflammatory' = '#FBE77C'), #"MISC_acute_validation" = '#00FFFF'),
Severity = c("-1" = "white", "0" = "white", "2" = "#efe5d7", "3" = "#bc8e52"),
group = c("validation" = "orange", "discovery" = "light blue","UCSF" = "light blue") #,
# PCR = c("1" = "green", "2" = "blue"),
# Antibody =c("1" = "green", "2" = "blue")
)

###----------------------------------
## Plot

heatmap_plt_clust <- pheatmap(mat,

         # Colors
         col=color,
         breaks=breaksList, 
         annotation_col=metadata,
         annotation_colors=my_colour,
         na_col = "#FFFFFF",

         # Fonts
         show_colnames=F,
         show_rownames=F,
         fontsize=12,
         fontsize_col=3,
         annotation_names_col=F,
         annotation_names_row=F,

         # Clustering

         clustering_distance_cols="correlation",
         clustering_distance_rows="correlation", 
         treeheight_row=0,
        treeheight_col= 15,

         # Misc.
         border_color=NA,
        legend=FALSE,
        annotation_legend=FALSE
        ) 

IDTH = 1.6
HEIGHT = 2.6

RES = 300


prefix = "ALL_CLUSTER"

png(file="plots/panelC_all.png",
        width=WIDTH,height=HEIGHT, units ="in", bg="white", res = RES, #useRaster = TRUE,
        fonts="Helvetica",  pointsize=6)

draw(heatmap_plt_clust,show_heatmap_legend=FALSE,padding = unit(0.25,"mm"))

dev.off()



png 
  2

---
## Biomarkers

In [30]:
get_exp_plot <- function(GENE,meta_data,count_cpm,gene_lookup= annotation){
    
    samp_df <- data.frame("cfrna_sample_id" = meta_data$cfrna_sample_id,
                      "Diagnosis" = meta_data$Diagnosis,
                     "severity" = meta_data$severity,
                     "group" = meta_data$group)
    
    GENE <- gene_lookup[which(gene_lookup$gene_name == GENE),]$gene_id

    samp_df <- merge(samp_df,count_cpm[,GENE],by.x="cfrna_sample_id",by.y=0)

    samp_df$plotting_name <- paste(samp_df$Diagnosis,samp_df$group,sep="\n")

    samp_df$plotting_name <- factor(samp_df$plotting_name, levels = c("Control_Non-inflammatory\ndiscovery","COVID-19\ndiscovery","MIS-C\ndiscovery","MIS-C\nvalidation"))

    samp_df %>%
    ggplot(aes(x=plotting_name,y=y,fill=plotting_name))+
    geom_boxplot(outlier.shape = NA)+
    geom_jitter(height=0,width=0.2,size = 1)+
    annotate(geom="text",label=gsub(".*_","",GENE),x=-Inf,y=Inf,hjust=-.1,vjust=1.25)+
    scale_fill_manual(values=color_groups)+
    theme_alex()+
    theme(legend.position = "none",
        axis.title.x = element_blank(),
        panel.grid.minor = element_blank(),
         plot.title = element_text(hjust = 0.5))+
    labs(y="CPM")+
#     labs(y="TPM",title=gsub(".*\\_","",GENE))+
    scale_x_discrete(guide = guide_axis(n.dodge = 2))
}


get_fig_plot <- function(GENE,meta_data,count_cpm,YLIM, gene_lookup= annotation, expGroupPalette=expGroupPalette){
    
    ##---------------------------------------
    # PREPARE DATA
    samp_df <- data.frame("cfrna_sample_id" = meta_data$cfrna_sample_id,
                      "Diagnosis" = meta_data$Diagnosis,
                     "severity" = meta_data$severity,
                        "group" = meta_data$group)
    GENE_N <- GENE
    GENE <- gene_lookup[which(gene_lookup$gene_name == GENE),]$gene_key

    samp_df <- merge(samp_df,count_cpm[,GENE],by.x="cfrna_sample_id",by.y=0)

    samp_df$plotting_name <- samp_df$Diagnosis
    
    samp_df$plotting_name <- ifelse(samp_df$Diagnosis == "MIS-C" & samp_df$group == "validation","MIS-C_validation",samp_df$plotting_name)

    samp_df$plotting_name <- factor(samp_df$plotting_name, levels = c("Control_Non-inflammatory","COVID-19","MIS-C","MIS-C_validation")) #,"MIS-C_validation"
    
    ##---------------------------------------
    # MAKE PLOT
    PLOT <- samp_df %>%
    ggplot(aes(x=plotting_name,y=y,color=plotting_name))+
#     geom_jitter(height=0,width=0.2,size = .65, color="black")+
    geom_point(  size = .75, position = position_jitter(seed = 42,height=0,width=0.2))+ #colour="black",pch=21,aes(fill=plotting_name),
#     annotate(geom="text",label=gsub(".*_","",GENE_N),x=-Inf,y=Inf,hjust=-.1,vjust=1.25,size = 2)+
    scale_color_manual(values=color_groups)+
    theme_prevail()+
    theme(legend.position = "none",
        axis.title.x = element_blank(),
        axis.title.y = element_blank(),
        axis.text.x = element_blank(),
        panel.grid.minor = element_blank(),
#         panel.grid.major = element_blank(),
#         panel.border = element_blank(),
        plot.title = element_text(hjust = 0.5),
         plot.margin=grid::unit(c(0.02,0.02,0,0), "in"))+
    labs(y="CPM")+
#     labs(y="TPM",title=gsub(".*\\_","",GENE))+
    scale_x_discrete(guide = guide_axis(n.dodge = 2)) #+
#     scale_color_manual(values=expGroupPalette)
    
    ##---------------------------------------
    # ADD ARROWS
    outliers <- samp_df%>% 
            filter(y > YLIM)
    
    print("num outliers:")
    print(nrow(outliers))

    y_stop = YLIM 
    y_start= YLIM - (YLIM*0.05)

    XORDER <- list('Control_Non-inflammatory'= 1,
                'COVID-19'= 2,
                'MIS-C'= 3,
                'MIS-C'= 4)

    offset_x = 1  # offset for different condition / plotting names
    offset_y = .20     # offset from top for different outliers in same condition


    text_offset = 0.09
    text_size = 1.5
    text_lineheight = .75

    arrow_size = 0.5

    for (i  in 1:length(XORDER)){
        group_name <- names(XORDER[i])
        group_outliers <- outliers %>% filter(plotting_name == group_name)

        ## ADD ARROW
        if (nrow(group_outliers) > 0){

            outlier_values <- round(group_outliers[, "y",drop=TRUE],2)
            outlier_values <- as.character(outlier_values[order(outlier_values,decreasing=TRUE)])

            text_center <- y_start * (1 - (.01*(length(outlier_values)-1)))

            XVAL <- XORDER[[group_name]]        

            PLOT <- PLOT + annotate("segment", x = XVAL, xend = XVAL,
                                              y = YLIM-0.007, yend = YLIM,
                                              size = arrow_size, lineend="butt", linejoin="mitre", arrow=arrow(length=unit(.06,"npc")))



            for (ii in 1:length(outlier_values)){

                text_color = "black"
                if (group_outliers[ii, "severity"] == 2){
                    text_color = "red"} 

                if (ii == 1){YVAL = YLIM
                            }else { 
                    YVAL = YLIM- ((ii-1)*0.006)
                }

                VAL <- outlier_values[ii]

                PLOT <- PLOT + annotate("text", x = XVAL, y = YVAL, hjust=-0.2,vjust=0.5,
                                              label = VAL,
                                              size = text_size, family = "Helvetica", lineheight = text_lineheight,color = text_color)

            }}}
    
    ##---------------------------------------
    # ADD SIG BARS
    stat.test <- samp_df %>%
    wilcox_test(y ~ plotting_name) %>%  #, ref.group = "MIS-C\ndiscovery"
    adjust_pvalue(method = "BH") %>% 
    add_significance("p.adj") %>% 
    add_xy_position(x = "plotting_name")
    
#     print(stat.test)
    
    stat.test <- stat.test %>% arrange(desc(y.position))
    stat.test$rank <- c(1:nrow(stat.test))
    stat.test$y.position <- YLIM - ((0.05*stat.test$rank)*YLIM)
    
    print(stat.test)
    PLOT <- PLOT + stat_pvalue_manual(stat.test, label = "p.adj.signif",tip.length=0) 
    
    return(PLOT)
}


In [32]:
keeper_columns <- c("gene_name","gene_type","baseMean","padj","log2FoldChange")
color_groups = c('COVID-19' = '#F0484E', 'MIS-C' = '#5CB2EB', 'Control_Non-inflammatory' = '#FBE77C', "MIS-C_validation" = "#2BC0B3")


PVAL_CUTOFF = 0.05
GROUPS = c("COVID-19", "Control_Non-inflammatory","MIS-C")



meta_data <- meta_data_all %>% 
    filter((Diagnosis %in% all_of(GROUPS)) & (timepoint %in% c("Not-hospitalized","acute"))) %>%
    mutate(cfrna_sample_id = wbrna_sample_id)


counts_cpm <- cpm_ftcount %>% t()

In [34]:
GENE = "ADAMTS2"

WIDTH = 1.0
HEIGHT = 1.5
YLIM = 410

pdf(file=paste0("plots/panelD_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,137,274,411)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 0
# A tibble: 6 × 14
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 y     MIS-C        MIS-C…    72    19      1183 1.13e- 6 2.26e- 6 ****        
2 y     COVID-19     MIS-C…    37    19       498 1.1 e- 2 1.32e- 2 *           
3 y     COVID-19     MIS-C     37    72      1086 1.16e- 1 1.16e- 1 ns          
4 y     Control_Non… MIS-C…    26    19        50 1.55e- 6 2.32e- 6 ****        
5 y     Control_Non… MIS-C     26    72        78 3.62e-12 2.17e-11 ****        
6 y     Control_Non… COVID…    26    37       108 6.92e- 8 2.08e- 7 ****        
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [35]:
GENE = "TRBV11-2"

WIDTH = 0.948
HEIGHT = 1.5
YLIM = 21

pdf(file=paste0("plots/panelD_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,7,14,21)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 0
# A tibble: 6 × 14
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 y     MIS-C          MIS-C…    72    19      791  2.97e-1 0.356   ns          
2 y     COVID-19       MIS-C…    37    19      240  5   e-2 0.1     ns          
3 y     COVID-19       MIS-C     37    72      757  2.12e-4 0.00127 **          
4 y     Control_Non-i… MIS-C…    26    19      239  8.63e-1 0.863   ns          
5 y     Control_Non-i… MIS-C     26    72      748  1.31e-1 0.196   ns          
6 y     Control_Non-i… COVID…    26    37      654. 1.5 e-2 0.045   *           
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [36]:
GENE = "SIGLEC1"

WIDTH = 1.0
HEIGHT = 1.5
YLIM = 660

pdf(file=paste0("plots/panelD_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,220,440,660)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 1
# A tibble: 6 × 14
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 y     MIS-C          MIS-C…    72    19     1028  7.82e-4 9.38e-4 ***         
2 y     COVID-19       MIS-C…    37    19      663  7.24e-8 2.17e-7 ****        
3 y     COVID-19       MIS-C     37    72     2199  2.93e-8 1.76e-7 ****        
4 y     Control_Non-i… MIS-C…    26    19      413  1.39e-4 2.08e-4 ***         
5 y     Control_Non-i… MIS-C     26    72     1076. 2.63e-1 2.63e-1 ns          
6 y     Control_Non-i… COVID…    26    37      170  5.48e-6 1.10e-5 ****        
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [37]:
GENE = "KLRB1"

WIDTH = 0.948
HEIGHT = 1.5
YLIM = 100

pdf(file=paste0("plots/panelD_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,33,66,99)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 2
# A tibble: 6 × 14
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 y     MIS-C        MIS-C…    72    19      772. 3.9 e- 1 4.35e- 1 ns          
2 y     COVID-19     MIS-C…    37    19      430. 1.8 e- 1 2.7 e- 1 ns          
3 y     COVID-19     MIS-C     37    72     1454. 4.35e- 1 4.35e- 1 ns          
4 y     Control_Non… MIS-C…    26    19      483  1.6 e-10 4.8 e-10 ****        
5 y     Control_Non… MIS-C     26    72     1775  1.5 e-11 9   e-11 ****        
6 y     Control_Non… COVID…    26    37      894  8.4 e- 9 1.68e- 8 ****        
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [38]:
GENE = "KLRF1"

WIDTH = 0.948
HEIGHT = 1.5
YLIM = 75

pdf(file=paste0("plots/SupPanelB_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,25,50,75)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 1
# A tibble: 6 × 14
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 y     MIS-C          MIS-C…    72    19     1028. 7.57e-4 9.08e-4 ***         
2 y     COVID-19       MIS-C…    37    19      582. 6.81e-5 1.36e-4 ***         
3 y     COVID-19       MIS-C     37    72     1736. 1   e-2 1   e-2 **          
4 y     Control_Non-i… MIS-C…    26    19      466. 5.07e-7 1.52e-6 ****        
5 y     Control_Non-i… MIS-C     26    72     1598. 1.03e-7 6.18e-7 ****        
6 y     Control_Non-i… COVID…    26    37      728. 5.93e-4 8.90e-4 ***         
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [39]:
GENE = "CD177"

WIDTH = 1.037
HEIGHT = 1.5
YLIM = 1800

pdf(file=paste0("plots/SupPanelB_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,600,1200,1800)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 0
# A tibble: 6 × 14
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 y     MIS-C        MIS-C…    72    19      451  2.3 e- 2 2.3 e- 2 *           
2 y     COVID-19     MIS-C…    37    19      114  1.52e- 5 2.28e- 5 ****        
3 y     COVID-19     MIS-C     37    72      834. 1   e- 3 1.2 e- 3 **          
4 y     Control_Non… MIS-C…    26    19        0  1.23e- 8 2.46e- 8 ****        
5 y     Control_Non… MIS-C     26    72       31  3.26e-13 1.96e-12 ****        
6 y     Control_Non… COVID…    26    37       40  7   e-10 2.1 e- 9 ****        
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [40]:
GENE = "ISG15"

WIDTH = 1.037
HEIGHT = 1.5
YLIM = 1200

pdf(file=paste0("plots/SupPanelB_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,400,800,1200)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 2
# A tibble: 6 × 14
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 y     MIS-C          MIS-C…    72    19       755 4.91e-1 7.36e-1 ns          
2 y     COVID-19       MIS-C…    37    19       577 4.6 e-5 9.2 e-5 ****        
3 y     COVID-19       MIS-C     37    72      2077 1.89e-6 5.67e-6 ****        
4 y     Control_Non-i… MIS-C…    26    19       259 7.94e-1 7.94e-1 ns          
5 y     Control_Non-i… MIS-C     26    72       874 6.21e-1 7.45e-1 ns          
6 y     Control_Non-i… COVID…    26    37       151 1.14e-6 5.67e-6 ****        
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

In [41]:
GENE = "CREB3L1"

WIDTH = 0.948
HEIGHT = 1.5
YLIM = 21

pdf(file=paste0("plots/SupPanelB_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

get_fig_plot(GENE,meta_data,counts_cpm,YLIM) + 
#     scale_y_continuous(breaks = scales::pretty_breaks(n = 4)) + 
    scale_y_continuous(breaks = c(0,7,14,21)) + 
    coord_cartesian(ylim = c(0,YLIM))

dev.off()

[1] "num outliers:"
[1] 0
# A tibble: 6 × 14
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 y     MIS-C          MIS-C…    72    19      473  3   e-2 3.6 e-2 *           
2 y     COVID-19       MIS-C…    37    19      156  1.04e-4 3.12e-4 ***         
3 y     COVID-19       MIS-C     37    72      885  1   e-3 1.5 e-3 **          
4 y     Control_Non-i… MIS-C…    26    19       96  5.33e-5 3.12e-4 ***         
5 y     Control_Non-i… MIS-C     26    72      575  1   e-3 1.5 e-3 **          
6 y     Control_Non-i… COVID…    26    37      440. 3.87e-1 3.87e-1 ns          
# … with 5 more variables: y.position <dbl>, groups <named list>, xmin <dbl>,
#   xmax <dbl>, rank <int>


png 
  2

---
## gene module scores

In [3]:
get_plots <- function(ct_counts,expGroupPalette,DiagnosisPalette,YLIM){

    
    ct_counts$timepoint <- factor(ct_counts$timepoint,levels=c("acute","post-acute","1m","3m+","Not-hospitalized"))

    common <- list(theme_prevail(),
        theme(legend.position = "none",
        axis.text.x = element_blank(),
        axis.title.y = element_blank(),
        axis.title.x = element_blank(),
#         panel.border = element_blank(),
        panel.grid.minor = element_blank()))

    box_common <- list(scale_x_discrete(expand = c(0, 0)),
        scale_fill_manual(values=DiagnosisPalette))


    ct_counts_SUM <- Rmisc::summarySE(ct_counts,
                 measurevar="value", 
                 groupvars=c("timepoint","Diagnosis")) %>% filter(N>1) %>% filter( !grepl("Control_Non-inflammatory",Diagnosis))

    MEAN_CNTRL <- mean(ct_counts[which(grepl("Control_Non-inflammatory",ct_counts$Diagnosis)),"value"])

    ##------------------------------------
    # Control Boxplot

    cntrl_boxplot <- ct_counts %>% 
    filter(grepl("Control",Diagnosis)) %>%
    ggplot(aes(x=Diagnosis,y=value,fill=expGroup))+
    geom_boxplot(width=0.5/4,size = 0.2,outlier.size = .01)+
    stat_summary(fun=mean, geom="point", shape=18, size=.2, color="white", fill="white") 

    ##------------------------------------
    # Acute Boxplot

acute_boxplot <- ct_counts %>% 
    filter(grepl("acute",timepoint) | grepl("Control_Non-inflammatory",Diagnosis)) %>%
#     mutate(expGroup = factor(expGroup,levels=c("Control_Non-inflammatory_discovery","MIS-C_validation","MIS-C_discovery","COVID-19_discovery")))%>%
    mutate(expGroup = factor(expGroup,levels=c("Control_Non-inflammatory","COVID-19","MIS-C","MIS-C_validation")))%>%
    ggplot(aes(x=expGroup,y=value))+
    geom_hline(yintercept = MEAN_CNTRL,alpha = 0.75, size = .5, linetype = "dashed",color = DiagnosisPalette[["Control_Non-inflammatory"]][1] )+
    geom_boxplot(aes(fill=expGroup), width=0.75,size = 0.2,outlier.size = .01, outlier.shape=NA)+
    stat_summary(fun=mean, geom="point", shape=18, size=.2, color="white", fill="white") 

    
    
    ##------------------------------------ 
    # LINE PLOT
    pd <- position_dodge(0.05)

    line_plot <- ct_counts_SUM %>%
    mutate(timepoint = factor(timepoint,levels=c("acute","post-acute","1m","3m+","Not-hospitalized"))) %>%
    ggplot(aes(x=timepoint, y=value, colour=Diagnosis, group=Diagnosis)) + 
        geom_hline(yintercept = MEAN_CNTRL,alpha = 0.75, size = .5, linetype = "dashed",color = DiagnosisPalette[["Control_Non-inflammatory"]][1] ) +
        geom_errorbar(aes(ymin=value-se, ymax=value+se), width=.1, position=pd) +
        geom_line(position=pd) +
        geom_point(position=pd, size = 0.5)+
    scale_x_discrete(expand = c(.1, 0))+
    scale_color_manual(values=DiagnosisPalette)

    ##------------------------------------
    # Customize

    # Calculate Y limits
    y_range_cntl <- layer_scales(cntrl_boxplot)$y$range$range
    y_range_acute <- layer_scales(acute_boxplot)$y$range$range
    y_range_ln <- layer_scales(line_plot)$y$range$range

    starts <- c(y_range_cntl[1],y_range_acute[1],y_range_ln[1],0)
    stops <- c(y_range_cntl[2],y_range_acute[2],y_range_ln[2])
    
    proper_scale <- list(coord_cartesian(ylim = c(min(starts), max(stops))), 
                    scale_y_continuous(labels = function(x) format(x, scientific = TRUE, digits = 2),
                                       breaks = seq(0,max(stops),length.out  = 4)))
    
    
    
    ##---------------------------------------
    # ADD SIG BARS
    
    stat.test <- ct_counts %>% filter(grepl("acute",timepoint) | grepl("Control_Non-inflammatory",Diagnosis)) %>%
    wilcox_test(value ~ expGroup, paired = FALSE) %>% 
    adjust_pvalue(method = "BH") %>% 
    add_significance("p") %>% 
    add_xy_position(x = "expGroup") 
    
    stat.test <- stat.test %>% arrange(desc(y.position))
    stat.test$rank <- c(1:nrow(stat.test))
    stat.test$y.position <- YLIM - ((0.05*stat.test$rank)*YLIM)
                                       
    sig_bars <- stat_pvalue_manual(stat.test, label = "p.adj.signif",tip.length=0) 
    
    acute_boxplot <- acute_boxplot + sig_bars
    
#     return(acute_boxplot)
    print(stat.test)
    
    ##---------------------------------------
    # FINALIZE
            
    line_plot <- line_plot + common + proper_scale
                                       
    cntrl_boxplot <- cntrl_boxplot + common + box_common + proper_scale
                        
    acute_boxplot <- acute_boxplot + common + box_common + proper_scale
    


    
    return(list("line_plot" = line_plot,
                "cntrl_boxplot" = cntrl_boxplot,
                "acute_boxplot" = acute_boxplot))
    
    }

DiagnosisPalette = c('#5CB2EB','#F0484E','#FBE77C','#2BC0B3')
names(DiagnosisPalette) <- c("MIS-C","COVID-19","Control_Non-inflammatory","MIS-C_validation")

expGroupPalette = c('#2BC0B3','#5CB2EB','#F0484E','#FBE77C')
names(expGroupPalette) <- c("MIS-C_validation","MIS-C_discovery","COVID-19_discovery","Control_Non-inflammatory_discovery")


In [8]:
GROUPS = c("COVID-19", "Control_Non-inflammatory","MIS-C")



meta_data <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") 

wb_ftcounts <- read.delim("../1_sample-data/wbrna_ftcounts.txt",row.names=1)  

wb_ftcounts <- wb_ftcounts[,colSums(wb_ftcounts) > 1]

#------------------------
## FILTER

gene.list <- read.delim("../0_support-files/genelist.remove.tsv",col.names = c("type,","ENSMBL","gene_symbol"))

gene.ids <- gsub("\\..*","",rownames(wb_ftcounts))

exclude.idx <- gene.ids %in% gene.list[,2]

wb_ftcounts = wb_ftcounts[!exclude.idx,] 

#------------------------
## NORMALIZE

wb_ftcounts <- edgeR::cpm(wb_ftcounts)

In [9]:
NOT_LEFT = list(theme(axis.text.y = element_blank(),axis.ticks.y=element_blank()))



# # ACUTE - LINE
lay <- rbind(c(2,2,1,1))
ACUTE_MARGIN = list(theme(plot.margin=grid::unit(c(0.02,-.025,0,0), "in")),scale_x_discrete(expand = expansion(add = 0.5)))
# ACUTE_MARGIN = theme(plot.margin=grid::unit(c(0,-.2,0,0), "in"))
CNTRL_MARGIN = theme(plot.margin=grid::unit(c(0,-.5,0,-.3), "in"))
LINE_MARGIN = theme(plot.margin=grid::unit(c(0.02,0.02,0,-0.025), "in"))
GRID_fun <- function(acute_boxplot, cntrl_boxplot, line_plot, lay){grid.arrange( line_plot, acute_boxplot, layout_matrix = lay)}


## get data frame with counts
tcr_ftcounts <- wb_ftcounts[grepl("TRBV11-2",rownames(wb_ftcounts)),] %>% data.frame()
colnames(tcr_ftcounts) <- "TRVB11.2"


ID_KEYS = c("wbrna_sample_id","Diagnosis","group","timepoint")
df <- merge(meta_data[,ID_KEYS], tcr_ftcounts, by.x= "wbrna_sample_id", by.y=0)
output_melt <- melt(df,id.vars = ID_KEYS) %>% mutate(Diagnosis_group = paste0(Diagnosis,"_",group)) %>% 
        mutate(expGroup = Diagnosis) %>% 
        mutate(expGroup = ifelse(group == "validation","MIS-C_validation",expGroup))

output_melt <- output_melt %>% filter(group %in% c("discovery","validation"))



Warning message in melt(df, id.vars = ID_KEYS):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(df). In the next version, this warning will become an error.”


In [10]:
WIDTH = 2.75
HEIGHT = 1.2

##------------------------------------
GENE = 'TRBV11-2'

LIM = 12


all_plots <- get_plots(output_melt,expGroupPalette,DiagnosisPalette,LIM)

LIMITS =  list(scale_y_continuous(breaks = seq(0,LIM,length.out  = 5)), #labels = function(x) format(x, scientific = TRUE, digits = 2)),
               coord_cartesian(ylim = c(0,LIM))
              )

cntrl_boxplot <- all_plots[["cntrl_boxplot"]] + CNTRL_MARGIN + NOT_LEFT + LIMITS
acute_boxplot <- all_plots[["acute_boxplot"]] + ACUTE_MARGIN + LIMITS
line_plot <- all_plots[["line_plot"]] + LINE_MARGIN + NOT_LEFT + LIMITS
               
pdf(file=paste0("plots/SupPanelC_",GENE,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

all_plt <- GRID_fun(acute_boxplot, cntrl_boxplot, line_plot, lay)

print(all_plt)
               
dev.off()



Warning message in qt(conf.interval/2 + 0.5, datac$N - 1):
“NaNs produced”


# A tibble: 6 × 15
  .y.   group1         group2    n1    n2 statistic       p   p.adj p.adj.signif
  <chr> <chr>          <chr>  <int> <int>     <dbl>   <dbl>   <dbl> <chr>       
1 value MIS-C          MIS-C…    84    19      917  3.12e-1 3.74e-1 ns          
2 value COVID-19       MIS-C…    50    19      315  2.9 e-2 5.8 e-2 ns          
3 value COVID-19       MIS-C     50    84     1172  1.71e-5 1.03e-4 ***         
4 value Control_Non-i… MIS-C…    26    19      239  8.63e-1 8.63e-1 ns          
5 value Control_Non-i… MIS-C     26    84      884. 1.42e-1 2.13e-1 ns          
6 value Control_Non-i… COVID…    26    50      903  5   e-3 1.5 e-2 *           
# … with 6 more variables: p.signif <chr>, y.position <dbl>,
#   groups <named list>, xmin <dbl>, xmax <dbl>, rank <int>


Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.



TableGrob (1 x 4) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,3-4) arrange gtable[layout]
2 2 (1-1,1-2) arrange gtable[layout]


png 
  2

In [11]:
GENE_NAME <- "KLRB1"


## get data frame with counts
tcr_ftcounts <- wb_ftcounts[grepl(GENE_NAME,rownames(wb_ftcounts)),] %>% data.frame()
colnames(tcr_ftcounts) <- GENE_NAME


ID_KEYS = c("wbrna_sample_id","Diagnosis","group","timepoint")
df <- merge(meta_data[,ID_KEYS], tcr_ftcounts, by.x= "wbrna_sample_id", by.y=0)
output_melt <- melt(df,id.vars = ID_KEYS) %>% mutate(Diagnosis_group = paste0(Diagnosis,"_",group)) %>% 
        mutate(expGroup = Diagnosis) %>% 
        mutate(expGroup = ifelse(group == "validation","MIS-C_validation",expGroup))
# output_melt <- output_melt %>% filter(group %in% c("discovery","validation"))


##-----------------------------------
LIM = 100
WIDTH = 2.75
HEIGHT = 1.2

all_plots <- get_plots(output_melt,expGroupPalette,DiagnosisPalette,LIM)

LIMITS =  list(scale_y_continuous(breaks = seq(0,LIM,length.out  = 5)), #labels = function(x) format(x, scientific = TRUE, digits = 2)),
               coord_cartesian(ylim = c(0,LIM))
              )

cntrl_boxplot <- all_plots[["cntrl_boxplot"]] + CNTRL_MARGIN + NOT_LEFT + LIMITS
acute_boxplot <- all_plots[["acute_boxplot"]] + ACUTE_MARGIN + LIMITS
line_plot <- all_plots[["line_plot"]] + LINE_MARGIN + NOT_LEFT + LIMITS
               
pdf(file=paste0("plots/SupPanelC_",GENE_NAME,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

all_plt <- GRID_fun(acute_boxplot, cntrl_boxplot, line_plot, lay)

print(all_plt)
               
dev.off()

Warning message in melt(df, id.vars = ID_KEYS):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(df). In the next version, this warning will become an error.”
Warning message in qt(conf.interval/2 + 0.5, datac$N - 1):
“NaNs produced”


# A tibble: 6 × 15
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 value MIS-C        MIS-C…    84    19       882 4.78e- 1 4.78e- 1 ns          
2 value COVID-19     MIS-C…    50    19       592 1.17e- 1 1.76e- 1 ns          
3 value COVID-19     MIS-C     50    84      2409 1.56e- 1 1.87e- 1 ns          
4 value Control_Non… MIS-C…    26    19       483 1.6 e-10 4.8 e-10 ****        
5 value Control_Non… MIS-C     26    84      2076 4.51e-12 2.71e-11 ****        
6 value Control_Non… COVID…    26    50      1213 7.26e-10 1.45e- 9 ****        
# … with 6 more variables: p.signif <chr>, y.position <dbl>,
#   groups <named list>, xmin <dbl>, xmax <dbl>, rank <int>


Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.



TableGrob (1 x 4) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,3-4) arrange gtable[layout]
2 2 (1-1,1-2) arrange gtable[layout]


png 
  2

In [12]:
GENE_NAME <- "ADAMTS2$"


## get data frame with counts
tcr_ftcounts <- wb_ftcounts[grepl(GENE_NAME,rownames(wb_ftcounts)),] %>% data.frame()
colnames(tcr_ftcounts) <- GENE_NAME


ID_KEYS = c("wbrna_sample_id","Diagnosis","group","timepoint")
df <- merge(meta_data[,ID_KEYS], tcr_ftcounts, by.x= "wbrna_sample_id", by.y=0)
output_melt <- melt(df,id.vars = ID_KEYS) %>% mutate(Diagnosis_group = paste0(Diagnosis,"_",group)) %>% 
        mutate(expGroup = Diagnosis) %>% 
        mutate(expGroup = ifelse(group == "validation","MIS-C_validation",expGroup))
# output_melt <- output_melt %>% filter(group %in% c("discovery","validation"))


##-----------------------------------
LIM = 120
WIDTH = 2.75
HEIGHT = 1.2

all_plots <- get_plots(output_melt,expGroupPalette,DiagnosisPalette,LIM)

LIMITS =  list(scale_y_continuous(breaks = seq(0,LIM,length.out  = 5)), #labels = function(x) format(x, scientific = TRUE, digits = 2)),
               coord_cartesian(ylim = c(0,LIM))
              )

cntrl_boxplot <- all_plots[["cntrl_boxplot"]] + CNTRL_MARGIN + NOT_LEFT + LIMITS
acute_boxplot <- all_plots[["acute_boxplot"]] + ACUTE_MARGIN + LIMITS
line_plot <- all_plots[["line_plot"]] + LINE_MARGIN + NOT_LEFT + LIMITS
               
pdf(file=paste0("plots/SupPanelC_",GENE_NAME,".pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

all_plt <- GRID_fun(acute_boxplot, cntrl_boxplot, line_plot, lay)

print(all_plt)
               
dev.off()

Warning message in melt(df, id.vars = ID_KEYS):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(df). In the next version, this warning will become an error.”
Warning message in qt(conf.interval/2 + 0.5, datac$N - 1):
“NaNs produced”


# A tibble: 6 × 15
  .y.   group1       group2    n1    n2 statistic        p    p.adj p.adj.signif
  <chr> <chr>        <chr>  <int> <int>     <dbl>    <dbl>    <dbl> <chr>       
1 value MIS-C        MIS-C…    84    19      1383 6.67e- 7 1.33e- 6 ****        
2 value COVID-19     MIS-C…    50    19       689 4   e- 3 4.8 e- 3 **          
3 value COVID-19     MIS-C     50    84      1658 4.2 e- 2 4.2 e- 2 *           
4 value Control_Non… MIS-C…    26    19        50 1.55e- 6 2.32e- 6 ****        
5 value Control_Non… MIS-C     26    84        94 1.67e-12 1.00e-11 ****        
6 value Control_Non… COVID…    26    50       142 1.19e- 8 3.57e- 8 ****        
# … with 6 more variables: p.signif <chr>, y.position <dbl>,
#   groups <named list>, xmin <dbl>, xmax <dbl>, rank <int>


Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.



TableGrob (1 x 4) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,3-4) arrange gtable[layout]
2 2 (1-1,1-2) arrange gtable[layout]


png 
  2